In [68]:
import json
import requests
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
# import torch
# import torch.nn as nn
# import torch

In [69]:
# Replace with your own path
technicals = True
normalize_by_coin = False

df = pd.read_csv('/Users/timwu0/Documents/CS329P/afterhours_crypto/preprocessing/crypto_data.csv')
coins = df['coin'].unique()
print(coins, len(coins))
    
#convert coin to one-hot vectors
for coin in coins:
    df[coin] = df['coin'] == coin



['BTC-USD' 'ETH-USD' 'SOL1-USD' 'BNB-USD' 'DOT1-USD' 'ADA-USD' 'XRP-USD'
 'DOGE-USD' 'SHIB-USD' 'AVAX-USD' 'LTC-USD' 'XMR-USD' 'ETC-USD' 'REP-USD'
 'MAID-USD' 'STEEM-USD'] 16


In [70]:
def relative_strength_idx(df, n):
    close = df['close']
    delta = close.diff()
    # delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rs.fillna(1.0, inplace=True)
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

def MACD(df, short=12, long=26):
    EMA_short = pd.Series(df['close'].ewm(span=short, min_periods=short).mean())
    EMA_long = pd.Series(df['close'].ewm(span=long, min_periods=long).mean())
    return EMA_short - EMA_long 
'''
if technicals:
    RSI = pd.Dataframe()
    MACD = pd.Dataframe()
    MACD_signal = pd.Dataframe()
    for coin in coins:
        RSI = pd.concat([RSI, relative_strength_idx(df.loc[df[coin]]).fillna(50)])
        df['MACD'] = pd.Series(MACD(df)).fillna(0)
        df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean()).fillna(0)
'''
if technicals:
    df['RSI_14'] = relative_strength_idx(df, 14).fillna(0)
    df['RSI_30'] = relative_strength_idx(df, 30).fillna(0)
    df['RSI_50'] = relative_strength_idx(df, 50).fillna(0)
    df['RSI_200'] = relative_strength_idx(df, 200).fillna(0)
    df['MACD'] = pd.Series(MACD(df)).fillna(0)
    df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean()).fillna(0)
df

,date,high,low,open,close,volume,adjclose,coin,p,BTC-USD,...,ETC-USD,REP-USD,MAID-USD,STEEM-USD,RSI_14,RSI_30,RSI_50,RSI_200,MACD,MACD_signal
0,1483228800,1003.080017,958.698975,963.658020,998.325012,147775008.0,998.325012,BTC-USD,1.035974,True,...,False,False,False,False,50.000000,50.000000,50.000000,50.000000,0.000000,0.000000
1,1483315200,1031.390015,996.702026,998.617004,1021.750000,222184992.0,1021.750000,BTC-USD,1.023165,True,...,False,False,False,False,50.000000,50.000000,50.000000,50.000000,0.000000,0.000000
2,1483401600,1044.079956,1021.599976,1021.599976,1043.839966,185168000.0,1043.839966,BTC-USD,1.021770,True,...,False,False,False,False,50.000000,50.000000,50.000000,50.000000,0.000000,0.000000
3,1483488000,1159.420044,1044.400024,1044.400024,1154.729980,344945984.0,1154.729980,BTC-USD,1.105640,True,...,False,False,False,False,50.000000,50.000000,50.000000,50.000000,0.000000,0.000000
4,1483574400,1191.099976,910.416992,1156.729980,1013.380005,510199008.0,1013.380005,BTC-USD,0.876073,True,...,False,False,False,False,50.000000,50.000000,50.000000,50.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22620,1637712000,0.673348,0.607530,0.621976,0.612812,27702150.0,0.612812,STEEM-USD,0.985266,False,...,False,False,False,True,50.727463,51.694534,51.613728,44.243898,0.002363,0.002923
22621,1637798400,0.695432,0.623388,0.643072,0.631901,45568311.0,0.631901,STEEM-USD,0.982629,False,...,False,False,False,True,54.625830,53.765226,48.011556,44.795767,0.003910,0.003120
22622,1637884800,0.819299,0.673438,0.683161,0.702681,204342166.0,0.702681,STEEM-USD,1.028573,False,...,False,False,False,True,66.873809,65.381573,53.115098,47.112916,0.010723,0.004641
22623,1637971200,0.734130,0.674832,0.703990,0.706345,38813453.0,0.706345,STEEM-USD,1.003345,False,...,False,False,False,True,66.795869,63.605807,52.857673,46.476107,0.016231,0.006959


In [71]:
def train_test_split(df, test_size, train_size):
    train_data = pd.DataFrame(columns=df.columns)
    test_data = pd.DataFrame(columns=df.columns)
    for coin in coins:
        df_coin = df.loc[df['coin'] == coin]
        
        split = int(train_size * len(df_coin))
        test_end = min(int((train_size + test_size) * len(df_coin)), len(df_coin))
        
        coin_train = df_coin.iloc[:split].copy()
        coin_test = df_coin.iloc[split:test_end].copy()
        if normalize_by_coin:
            mean_p = coin_train['p'].mean()
            std_p = coin_train['p'].std()
            coin_train['p'] = ((coin_train['p']-mean_p)/std_p)#.round(1)
            coin_test['p'] = ((coin_test['p']-mean_p)/std_p)#.round(1)

        train_data = pd.concat([train_data, coin_train]) 
        test_data = pd.concat([test_data, coin_test]) 
            
    
    # print(train_data.columns)
    if not normalize_by_coin:
        mean_p = train_data['p'].mean()
        std_p = train_data['p'].std()
        train_data['p'] = ((train_data['p']-mean_p)/std_p)#.round(1)
        test_data['p'] = ((test_data['p']-mean_p)/std_p)#.round(1)


    return train_data.drop(['coin'], axis=1), test_data.drop(['coin'], axis=1)
train, test = train_test_split(df, 0.15, 0.5)
print(train.shape, test.shape)

(11307, 30) (3388, 30)


In [72]:
window_len = 22
test_size = 0.2

def extract_window_data(df, window_len):
    window_data = []
    
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        
        window_data.append(tmp.values)
    return np.array(window_data)

def prepare_data(df, target_col, window_len, test_size, train_size):
    train_data, test_data = train_test_split(df, test_size=test_size, train_size=train_size)
    # print(train_data.shape)
    x_train = extract_window_data(train_data, window_len)
    # print(X_train.shape)
    x_test = extract_window_data(test_data, window_len)
    
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values

    #print(len(train_data[target_col][:-window_len].values-1))
    #print(len(y_train))
    
    # print(X_train, y_train)

    return train_data, test_data, x_train, x_test, y_train, y_test


In [73]:
epochs=3
loss='mse'
optimizer = 'adam'
dropout=0.5
num_layers=4
cell_size=8
dense_units=100
technicals=False
model_type='rnn'
neurons=100

In [74]:
folds=5
min_train_size=0.4
test_size=0.15

train_sizes = []
for k in range(folds):
    train_sizes.append(min_train_size + k / (folds - 1) * (1 - (test_size + min_train_size)))

In [75]:
from rnn_model import rnn
from lstm_model import lstm

# min_MSE = 999# np.float(inf)
# min_MSE_k = 0
MSE_sum = 0
MAE_sum = 0
for i, train_size in enumerate(train_sizes):
    train, test, x_train, x_test, y_train, y_test = prepare_data(df, 'p', window_len=window_len, test_size=test_size, train_size=train_size)
    features = tf.convert_to_tensor(x_train, dtype=tf.float32)
    labels = tf.expand_dims(tf.convert_to_tensor(y_train, dtype=tf.float32), axis=1)
    print('Size of fold train data:', features.shape, labels.shape)
    if model_type == 'rnn':
        model = rnn(features=features, 
            labels=labels, 
            dropout=dropout, 
            num_layers=num_layers, 
            cell_size=cell_size, 
            dense_units=dense_units,
            technicals=technicals)
    elif model_type == 'lstm':
        model = lstm(features=features, 
            labels=labels, 
            dropout=dropout, 
            neurons=neurons, 
            dense_units=dense_units,
            technicals=technicals)
    model.fit(features, labels, epochs=epochs, shuffle=True)
    preds = model.predict(tf.convert_to_tensor(x_test, dtype=tf.float32))
    MSE = ((preds - y_test) ** 2).mean()
    MAE = np.abs(preds - y_test).mean()
    print('preds:', preds.squeeze()[:10])
    #if MSE < min_MSE:
    #    min_MSE = min(min_MSE, MSE)
    #    min_MSE_k = i + 1
    MSE_sum += MSE 
    MAE_sum += MAE
        
    print('Fold number', i + 1, 'MSE: ', MSE, 'MAE: ', MAE)
print('MSE: ', MSE_sum/len(train_sizes))
print('MAE:', MAE_sum/len(train_sizes))
    

Size of fold train data: (9020, 22, 30) (9020, 1)
Epoch 1/3
282/282 [==============================] - 14s 26ms/step - loss: 1.0268
Epoch 2/3
282/282 [==============================] - 9s 31ms/step - loss: 1.0017
Epoch 3/3
282/282 [==============================] - 9s 32ms/step - loss: 1.0013
preds: [0.0093545 0.0093545 0.0093545 0.0093545 0.0093545 0.0093545 0.0093545
 0.0093545 0.0093545 0.0093545]
Fold number 1 MSE:  0.3725640251548274 MAE:  0.41436927404063256
Size of fold train data: (11562, 22, 30) (11562, 1)
Epoch 1/3
362/362 [==============================] - 13s 28ms/step - loss: 1.0187
Epoch 2/3
362/362 [==============================] - 9s 26ms/step - loss: 1.0011
Epoch 3/3
362/362 [==============================] - 10s 26ms/step - loss: 1.0008
preds: [0.00676203 0.00676203 0.00676203 0.00676203 0.00676203 0.00676203
 0.00676203 0.00676203 0.00676203 0.00676203]
Fold number 2 MSE:  0.5780582664181897 MAE:  0.4716267462571408
Size of fold train data: (14115, 22, 30) (14115, 1

In [59]:
preds = model.predict(tf.convert_to_tensor(x_test, dtype=tf.float32))

print(preds[-20: ])
print(y_test[:20])

print(y_test.mean(), preds.mean(), (y_test - preds).mean())

print('MAE: ', (np.absolute(preds - y_test)).mean())
print('MSE: ', ((preds - y_test) ** 2).mean())


[[0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]
 [0.03487068]]
[-0.06149392 -0.05749807 -1.10694917 -0.02561842 -0.15050931 -0.40961858
  0.4243599  -0.30243512  0.00781457  0.01770343 -0.13301794  0.17741659
 -0.46210356 -0.14812357  0.02879439 -0.13524013 -0.01463505 -0.09335511
 -0.05945663 -0.13619578]
-0.03512793039011136 0.03520042 -0.07032835117598925
MAE:  0.41952661693636795
MSE:  0.3755693947522699
